In [44]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.notebook import tqdm


In [2]:
# set here globally
exchange = "ise"
models = "classical"
subset = "test" # "all"
strategy = "supervised"

key = f"{exchange}_{models}_{strategy}_{subset}"

In [3]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

dataset = f"fbv/thesis/{exchange}_{strategy}_unscaled:latest"
results = f"fbv/thesis/{key}:latest"

# load unscaled data
artifact = run.use_artifact(dataset) # type: ignore
data_dir = artifact.download()

# load results
artifact = run.use_artifact(results) # type: ignore
results_dir = artifact.download()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_supervised_unscaled:latest, 3391.53MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_classical_supervised_test:latest, 63.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type"
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set_extended_60"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set_extended_20"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set_extended_20"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train,val,test])
    del train, val, test
    
elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set_extended_20"), engine="fastparquet", columns=columns
    )

results_data = pd.read_parquet(
    Path(results_dir, "results"), engine="fastparquet"
)


assert len(eval_data) == len(results_data)

X_print = eval_data


### Robustness Checks

In [5]:
X_print.head()

,buy_sell,EXPIRATION,QUOTE_DATETIME,TRADE_SIZE,TRADE_PRICE,ask_ex,bid_ex,myn,OPTION_TYPE,issue_type
39342171,-1,2015-12-18,2015-11-06 09:30:00,2,0.520000,0.600000,0.520000,0.921659,P,%
39342172,-1,2015-11-27,2015-11-06 09:30:00,1,7.820000,8.150000,7.600000,1.001696,C,0
39342173,-1,2017-01-20,2015-11-06 09:30:00,1,28.889999,32.049999,28.799999,0.886115,C,0
39342174,1,2015-11-20,2015-11-06 09:30:00,1,2.250000,2.250000,1.850000,1.009261,C,%
39342175,-1,2015-12-18,2015-11-06 09:30:00,1,1.700000,1.950000,1.700000,0.991880,C,%


In [6]:

# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map({'0': 'Stock options', 'A': 'Index option', '7': 'Others', 'F':'Others', '%': 'Others', ' ': 'Others'})

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "myn <= 0.7",
    "myn (0.7-0.9]",
    "myn (0.9-1.1]",
    "myn (1.1-1.3]",
    "myn > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# proximity to quotes
mid_ex = 0.5 * (X_print["ask_ex"] + X_print["bid_ex"])
spread_ex = X_print["ask_ex"] - X_print["bid_ex"]
X_print["prox_q"] = (X_print["TRADE_PRICE"] - mid_ex) / (0.5 * spread_ex)
X_print["mid"] = mid_ex

# p. 31
def map_quotes(x):
    abs_x = np.abs(x)
    if 1 < x < np.inf:
        return "outside"
    elif x == 0:
        return "at mid"
    elif x == 1:
        return "at quote"
    return "inside"
    
X_print["prox_q_binned"] = X_print["prox_q"].apply(map_quotes)

# clean up empty buckets, as it causes empty grouping in result set generatio
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()


X_print.drop(columns=["EXPIRATION","QUOTE_DATETIME", "TRADE_SIZE", "ttm", "myn", "prox_q", "ask_ex", "bid_ex", "year"], inplace=True)


In [7]:
X_print.head(20)

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,mid,prox_q_binned
39342171,-1,0.520000,P,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],0.560000,inside
39342172,-1,7.820000,C,Stock options,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],7.875000,inside
39342173,-1,28.889999,C,Stock options,"(0,1]",2015,ttm > 12 month,myn (0.7-0.9],30.424999,inside
39342174,1,2.250000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],2.050000,outside
39342175,-1,1.700000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],1.825000,inside
39342176,1,0.350000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.280000,inside
39342177,1,0.440000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.360000,inside
39342178,-1,0.720000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.810000,inside
39342179,1,11.300000,C,Stock options,"(5,11]",2015,ttm > 12 month,myn > 1.3,NaN,inside
39342180,1,2.190000,P,Stock options,"(1,3]",2015,ttm > 12 month,myn (0.7-0.9],1.980000,inside


In [8]:
X_print = pd.concat([X_print, results_data], axis=1)

## Results Set Generation

In [89]:
LUT = {"(ex)": " (ex)", 
       "(best)": " (best)",
       "rev_": "Rev. ",
       "tick": "Tick",
       "quote": "Quote",
       "trade_size": "Trade Size",
       "depth": "Depth",
       "->": " $\\to$ ",
       "lr": "\gls{LR}",
       "emo": "\gls{EMO}", 
       "clnv": "\gls{CLNV}",
      "OPTION_TYPE": "Option Type",
      "(":"$(", # put interval start in math env
      "]":"]$", # put interval end in math env
      "_":"$\_"
      }
       
def cell_str(x):
    for orig, sub in LUT.items():
        x = x.replace(orig,sub)
    return x

In [22]:
def set_tex_style(styler, caption, label):
    res = (
    styler.set_caption(caption)
    # .hide(axis="index")
    .format_index(cell_str, axis = 0)
    .format_index(cell_str, axis = 1)
    .format(precision=4, decimal='.', thousands=",", escape=False, hyperlinks=None)
    .to_latex(f"{label}.tex", siunitx=True, position_float="centering", hrules=True, clines="skip-last;data",
                      label="tab:"+label, caption=caption)
             )
    return res

In [90]:
classifiers = results_data.columns.tolist()
criterions = ["OPTION_TYPE", "issue_type", "TRADE_SIZE_binned", "year_binned","ttm_binned","myn_binned","prox_q_binned"]

## Accurcay Calculation

In [24]:
result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(X_print["buy_sell"], X_print[classifier])
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(lambda x: accuracy_score(x["buy_sell"], x[classifier]))
            .mul(100)
            .rename(classifier)
            )
        res.loc["all"] = acc_tot * 100
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(set_tex_style, caption=(f"long-tbd","short-tbd"), label=f"{key.lower()}-{criterion.lower()}")

    # store all result sets for later use
    result_dfs.append(result_df)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [32]:
result_dfs[0]

OPTION_TYPE,C,P,all
tick(all),53.573900,53.078335,53.342559
tick(ex),50.372016,50.097673,50.243947
quote(best),58.711418,60.582925,59.585081
quote(ex),56.344838,57.740199,56.996225
lr(ex),56.418422,57.801998,57.064307
lr(best),58.774523,60.594004,59.623898
rev_lr(ex),56.486528,57.888561,57.141029
rev_lr(best),58.792914,60.650285,59.659977
emo(ex),53.505205,54.111720,53.788340
emo(best),55.509078,56.409156,55.929255


In [33]:
result_dfs[1]

issue_type,Index option,Others,Stock options,all
tick(all),51.554245,53.257312,53.404570,53.342559
tick(ex),50.871870,49.870976,50.384689,50.243947
quote(best),57.814700,64.962501,57.447408,59.585081
quote(ex),53.720453,62.409791,54.866364,56.996225
lr(ex),53.782831,62.501336,54.925098,57.064307
lr(best),57.820371,64.982376,57.494341,59.623898
rev_lr(ex),53.667527,62.607179,54.993031,57.141029
rev_lr(best),57.820371,65.030275,57.526223,59.659977
emo(ex),51.285832,57.505227,52.329176,53.788340
emo(best),51.291502,60.096614,54.321831,55.929255


In [34]:
result_dfs[2]

TRADE_SIZE_binned,"(0,1]","(1,3]","(3,5]","(5,11]",>11,all
tick(all),52.848691,52.830617,52.411300,53.630853,55.550232,53.342559
tick(ex),50.289740,50.222920,49.648294,50.186182,50.750727,50.243947
quote(best),58.064698,57.950477,58.330811,62.096424,63.258972,59.585081
quote(ex),55.290135,55.307640,55.736591,59.890120,60.706061,56.996225
lr(ex),55.423031,55.397594,55.774439,59.902085,60.697694,57.064307
lr(best),58.131272,57.984292,58.363067,62.118031,63.271784,59.623898
rev_lr(ex),55.480578,55.452455,55.932575,59.987698,60.765154,57.141029
rev_lr(best),58.176407,58.033288,58.418537,62.148640,63.260737,59.659977
emo(ex),51.933208,51.950476,52.814530,57.157071,57.237072,53.788340
emo(best),54.044949,54.083813,54.966063,59.130466,59.628784,55.929255


In [35]:
result_dfs[3]

year_binned,2015,2016,2017,all
tick(all),52.788691,53.281666,53.670502,53.342559
tick(ex),50.381045,50.183110,50.339021,50.243947
quote(best),56.015922,59.729647,60.451627,59.585081
quote(ex),54.878241,57.578845,56.356885,56.996225
lr(ex),54.887934,57.639197,56.462584,57.064307
lr(best),56.014491,59.765951,60.509837,59.623898
rev_lr(ex),54.995435,57.719699,56.520163,57.141029
rev_lr(best),56.094015,59.811811,60.508576,59.659977
emo(ex),52.835172,54.268153,52.995354,53.788340
emo(best),53.673481,56.083783,56.330285,55.929255


In [36]:
result_dfs[4]

ttm_binned,ttm <= 1 month,ttm (1-2] month,ttm (2-3] month,ttm (3-6] month,ttm (6-12] month,ttm > 12 month,all
tick(all),53.070524,53.404782,53.689628,53.882093,54.278666,54.903416,53.342559
tick(ex),50.124143,50.234919,50.465958,50.823732,50.631747,50.270019,50.243947
quote(best),60.417557,60.309049,59.459459,57.685187,57.463633,50.800908,59.585081
quote(ex),57.307837,57.732111,57.093101,55.978766,56.419706,52.670582,56.996225
lr(ex),57.366438,57.765969,57.177345,56.112714,56.473250,52.850764,57.064307
lr(best),60.438450,60.311838,59.502210,57.790397,57.546824,51.028867,59.623898
rev_lr(ex),57.469937,57.832292,57.172525,56.099023,56.494991,52.866520,57.141029
rev_lr(best),60.505662,60.322195,59.447095,57.750558,57.490944,51.003962,59.659977
emo(ex),54.413938,53.550872,52.892041,52.026183,51.948605,51.190243,53.788340
emo(best),56.810467,56.021255,54.903822,53.498997,52.866938,51.452765,55.929255


In [37]:
result_dfs[5]

myn_binned,myn <= 0.7,myn (0.7-0.9],myn (0.9-1.1],myn (1.1-1.3],myn > 1.3,all
tick(all),54.477316,55.496913,52.948661,51.638894,52.034053,53.342559
tick(ex),49.538611,50.675156,50.186938,50.100022,50.042366,50.243947
quote(best),61.259580,63.394870,60.045450,50.023348,48.772913,59.585081
quote(ex),60.365344,60.374359,57.084807,49.931615,48.710672,56.996225
lr(ex),60.428423,60.574909,57.115398,49.995855,48.819439,57.064307
lr(best),61.388021,63.545310,60.040269,50.099607,48.903247,59.623898
rev_lr(ex),60.572848,60.548998,57.214293,50.109692,48.858878,57.141029
rev_lr(best),61.458235,63.472991,60.103392,50.166887,48.882295,59.659977
emo(ex),58.223516,57.563791,53.144117,49.876078,50.041134,53.788340
emo(best),59.487948,59.872819,55.577691,50.262765,50.184718,55.929255


In [38]:
result_dfs[6]

prox_q_binned,at mid,at quote,inside,outside,all
tick(all),51.026223,53.438907,53.556692,55.386836,53.342559
tick(ex),49.213320,45.837103,50.762235,47.280674,50.243947
quote(best),55.901058,59.749921,60.019153,59.826754,59.585081
quote(ex),49.985180,59.519436,57.605810,59.585080,56.996225
lr(ex),50.625876,59.519436,57.610258,59.585080,57.064307
lr(best),56.239301,59.893951,60.009859,59.998813,59.623898
rev_lr(ex),51.205073,59.519436,57.632824,59.585080,57.141029
rev_lr(best),56.682999,59.751566,60.013477,59.858002,59.659977
emo(ex),51.026223,59.519436,53.558564,59.556997,53.788340
emo(best),55.385577,59.260986,55.668726,59.168183,55.929255


In [39]:
print_keys = ["Option Type", "Security Type", "Trade Size", "Year", "Time to Maturity", "Moneyness","Location to Quote"]

master = pd.concat(result_dfs, axis=1, keys=print_keys).T

In [40]:
master.iloc[:,0:10]

tick(all)   tick(ex)  quote(best)  \
Option Type       C                 53.573900  50.372016    58.711418   
                  P                 53.078335  50.097673    60.582925   
                  all               53.342559  50.243947    59.585081   
Security Type     Index option      51.554245  50.871870    57.814700   
                  Others            53.257312  49.870976    64.962501   
                  Stock options     53.404570  50.384689    57.447408   
                  all               53.342559  50.243947    59.585081   
Trade Size        (0,1]             52.848691  50.289740    58.064698   
                  (1,3]             52.830617  50.222920    57.950477   
                  (3,5]             52.411300  49.648294    58.330811   
                  (5,11]            53.630853  50.186182    62.096424   
                  >11               55.550232  50.750727    63.258972   
                  all               53.342559  50.243947    59.585081   
Year              2015              52.788691  50.381045    56.015922   
                  2016              53.281666  50.183110    59.729647   
                  2017              53.670502  50.339021    60.451627   
                  all               53.342559  50.243947    59.585081   
Time to Maturity  ttm <= 1 month    53.070524  50.124143    60.417557   
                  ttm (1-2] month   53.404782  50.234919    60.309049   
                  ttm (2-3] month   53.689628  50.465958    59.459459   
                  ttm (3-6] month   53.882093  50.823732    57.685187   
                  ttm (6-12] month  54.278666  50.631747    57.463633   
                  ttm > 12 month    54.903416  50.270019    50.800908   
                  all               53.342559  50.243947    59.585081   
Moneyness         myn <= 0.7        54.477316  49.538611    61.259580   
                  myn (0.7-0.9]     55.496913  50.675156    63.394870   
                  myn (0.9-1.1]     52.948661  50.186938    60.045450   
                  myn (1.1-1.3]     51.638894  50.100022    50.023348   
                  myn > 1.3         52.034053  50.042366    48.772913   
                  all               53.342559  50.243947    59.585081   
Location to Quote at mid            51.026223  49.213320    55.901058   
                  at quote          53.438907  45.837103    59.749921   
                  inside            53.556692  50.762235    60.019153   
                  outside           55.386836  47.280674    59.826754   
                  all               53.342559  50.243947    59.585081   

                                    quote(ex)     lr(ex)   lr(best)  \
Option Type       C                 56.344838  56.418422  58.774523   
                  P                 57.740199  57.801998  60.594004   
                  all               56.996225  57.064307  59.623898   
Security Type     Index option      53.720453  53.782831  57.820371   
                  Others            62.409791  62.501336  64.982376   
                  Stock options     54.866364  54.925098  57.494341   
                  all               56.996225  57.064307  59.623898   
Trade Size        (0,1]             55.290135  55.423031  58.131272   
                  (1,3]             55.307640  55.397594  57.984292   
                  (3,5]             55.736591  55.774439  58.363067   
                  (5,11]            59.890120  59.902085  62.118031   
                  >11               60.706061  60.697694  63.271784   
                  all               56.996225  57.064307  59.623898   
Year              2015              54.878241  54.887934  56.014491   
                  2016              57.578845  57.639197  59.765951   
                  2017              56.356885  56.462584  60.509837   
                  all               56.996225  57.064307  59.623898   
Time to Maturity  ttm <= 1 month    57.307837  57.366438  60.438450   
                  ttm (1-2] month   57.732111  57.765969  

In [41]:
master.iloc[:,11:-1]

rev_emo(best)   clnv(ex)  clnv(best)  \
Option Type       C                     55.606796  54.339938   56.597578   
                  P                     56.918757  55.220002   57.624942   
                  all                   56.219249  54.750772   57.077175   
Security Type     Index option          50.225410  51.741378   51.565586   
                  Others                60.691923  59.061310   61.574413   
                  Stock options         54.510022  53.060187   55.350035   
                  all                   56.219249  54.750772   57.077175   
Trade Size        (0,1]                 54.527170  52.960619   55.325992   
                  (1,3]                 54.604379  52.860507   55.227950   
                  (3,5]                 55.583435  53.603599   55.942535   
                  (5,11]                59.395087  57.970916   60.052111   
                  >11                   58.948426  58.451558   60.905240   
                  all                   56.219249  54.750772   57.077175   
Year              2015                  53.896634  53.310115   54.346464   
                  2016                  56.331248  55.220189   57.213552   
                  2017                  56.741540  54.146126   57.680321   
                  all                   56.219249  54.750772   57.077175   
Time to Maturity  ttm <= 1 month        57.389778  55.218753   57.896591   
                  ttm (1-2] month       56.123727  54.781831   57.153726   
                  ttm (2-3] month       54.514668  54.139345   56.233903   
                  ttm (3-6] month       52.877975  53.356785   54.963670   
                  ttm (6-12] month      52.287837  53.572892   54.608195   
                  ttm > 12 month        51.284273  52.078701   51.966881   
                  all                   56.219249  54.750772   57.077175   
Moneyness         myn <= 0.7            59.920652  58.963623   60.591971   
                  myn (0.7-0.9]         59.437301  58.610295   61.213353   
                  myn (0.9-1.1]         55.936957  54.333552   56.890708   
                  myn (1.1-1.3]         51.443547  49.670923   50.240246   
                  myn > 1.3             50.924822  48.758431   49.565089   
                  all                   56.219249  54.750772   57.077175   
Location to Quote at mid                56.061055  51.026223   55.668169   
                  at quote              59.204507  59.519436   59.225161   
                  inside                55.944149  54.734732   57.039733   
                  outside               59.160668  59.555810   59.151966   
                  all                   56.219249  54.750772   57.077175   

                                    rev_clnv(ex)  rev_clnv(best)  \
Option Type       C                    54.463541       56.778181   
                  P                    55.789794       58.177595   
                  all                  55.082666       57.431459   
Security Type     Index option         50.663945       50.566598   
                  Others               59.616119       62.156604   
                  Stock options        53.324088       55.633455   
                  all                  55.082666       57.431459   
Trade Size        (0,1]                53.402249       55.845825   
                  (1,3]                53.345081       55.772635   
                  (3,5]                54.319270       56.576993   
                  (5,11]               58.314762       60.371504   
                  >11                  58.002671       60.451319   
                  all                  55.082666       57.431459   
Year              2015                 53.554966       54.642422   
                  2016                 55.534174       57.538530   
                  2017                 54.548922       58.122295   
                  all                  55.082666       57.431459   
Time to Maturity  ttm <= 1 month       55.802325       58.518490   
       

In [42]:
master.style.pipe(set_tex_style, caption=("master-short","master-long"), label=f"{key}-master")

## Effective Spread

In [91]:
eff_dfs = []

classifiers.extend(["buy_sell"])

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (X_print.groupby([criterion])[["TRADE_PRICE", "mid", classifier]]
            .apply(lambda x: pd.Series({'nominal':  effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal"), 
                                        'rel': effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")}) 
            )
            .mul(100))
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(set_tex_style, caption=(f"long-tbd","short-tbd"), label=f"{key.lower()}-{criterion.lower()}-eff-spread")

    # store all result sets for later use
    eff_dfs.append(result_df)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [92]:
eff_dfs[0]

OPTION_TYPE                                                         C  \
tick(all)                                          nominal   2.368956   
                                                   rel       2.787060   
tick(ex)                                           nominal   1.641275   
                                                   rel       1.216144   
quote(best)                                        nominal   6.277463   
                                                   rel       8.687091   
quote(ex)                                          nominal  17.148075   
                                                   rel      15.908621   
lr(ex)                                             nominal  17.148075   
                                                   rel      15.908621   
lr(best)                                           nominal   6.328119   
                                                   rel       8.781897   
rev_lr(ex)                                         nominal  17.148075   
                                                   rel      15.908621   
rev_lr(best)                                       nominal   6.299029   
                                                   rel       8.741034   
emo(ex)                                            nominal   5.039973   
                                                   rel       8.211026   
emo(best)                                          nominal   4.732560   
                                                   rel       7.327618   
rev_emo(ex)                                        nominal   4.578047   
                                                   rel       7.694203   
rev_emo(best)                                      nominal   4.300934   
                                                   rel       7.000756   
clnv(ex)                                           nominal  12.215973   
                                                   rel      13.033426   
clnv(best)                                         nominal   5.545472   
                                                   rel       7.958071   
rev_clnv(ex)                                       nominal  12.070978   
                                                   rel      12.910730   
rev_clnv(best)                                     nominal   5.384427   
                                                   rel       7.822602   
trade_size(ex)->tick(all)                          nominal  -0.760849   
                                                   rel       0.157075   
trade_size(ex)->quote(best)                        nominal   1.015701   
                                                   rel       3.987788   
trade_size(ex)->quote(best)->quote(ex)             nominal   1.392459   
                                                   rel       4.309526   
quote(best)->quote(ex)                             nominal   6.751152   
                                                   rel       9.072622   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   1.392459   
                                                   rel       4.309526   
buy_sell                                           nominal   0.468056   
                                                   rel       3.207661   

OPTION_TYPE                                                         P  
tick(all)                                          nominal   1.967647  
                                                   rel       2.166616  
tick(ex)                                           nominal   1.472416  
                                                   rel       0.923338  
quote(best)                                        nominal   5.130346  
                                                   rel       9.340715  
quote(ex)                                          nominal  15.446562  
                                                   rel      16.586722  
lr(ex)                                             nominal  15.446

In [93]:
eff_dfs[1]

issue_type                                                  Index option  \
tick(all)                                          nominal      3.465315   
                                                   rel          0.707419   
tick(ex)                                           nominal      2.406263   
                                                   rel          0.604896   
quote(best)                                        nominal     10.262529   
                                                   rel          2.358332   
quote(ex)                                          nominal     56.374215   
                                                   rel          7.288216   
lr(ex)                                             nominal     56.374215   
                                                   rel          7.288216   
lr(best)                                           nominal     10.464668   
                                                   rel          2.369522   
rev_lr(ex)                                         nominal     56.374215   
                                                   rel          7.288216   
rev_lr(best)                                       nominal     10.448689   
                                                   rel          2.358542   
emo(ex)                                            nominal      6.077095   
                                                   rel          1.726999   
emo(best)                                          nominal      6.041168   
                                                   rel          1.715103   
rev_emo(ex)                                        nominal      5.573851   
                                                   rel          1.214250   
rev_emo(best)                                      nominal      5.524196   
                                                   rel          1.221930   
clnv(ex)                                           nominal     18.198776   
                                                   rel          4.135419   
clnv(best)                                         nominal      6.693394   
                                                   rel          1.704620   
rev_clnv(ex)                                       nominal     18.807074   
                                                   rel          4.162820   
rev_clnv(best)                                     nominal      7.078323   
                                                   rel          1.311818   
trade_size(ex)->tick(all)                          nominal      2.423847   
                                                   rel          0.589909   
trade_size(ex)->quote(best)                        nominal      7.154106   
                                                   rel          1.653495   
trade_size(ex)->quote(best)->quote(ex)             nominal      8.559894   
                                                   rel          1.749184   
quote(best)->quote(ex)                             nominal     12.245231   
                                                   rel          2.481747   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal      8.559894   
                                                   rel          1.749184   
buy_sell                                           nominal      3.341933   
                                                   rel          0.407809   

issue_type                                                     Others  \
tick(all)                                          nominal   1.370533   
                                                   rel       1.004399   
tick(ex)                                           nominal   0.824128   
                                                   rel      -0.777969   
quote(best)                                        nominal   4.259548   
                                                   rel      12.803767   
quote(ex)                                          nominal  10.49505

In [94]:
eff_dfs[2]

TRADE_SIZE_binned                                               (0,1]  \
tick(all)                                          nominal   2.502912   
                                                   rel       2.898426   
tick(ex)                                           nominal   1.796003   
                                                   rel       1.660074   
quote(best)                                        nominal   6.637340   
                                                   rel       7.585467   
quote(ex)                                          nominal  18.754145   
                                                   rel      14.979287   
lr(ex)                                             nominal  18.754145   
                                                   rel      14.979287   
lr(best)                                           nominal   6.695494   
                                                   rel       7.685510   
rev_lr(ex)                                         nominal  18.754145   
                                                   rel      14.979287   
rev_lr(best)                                       nominal   6.660040   
                                                   rel       7.646301   
emo(ex)                                            nominal   5.145619   
                                                   rel       7.060154   
emo(best)                                          nominal   4.801871   
                                                   rel       6.182700   
rev_emo(ex)                                        nominal   4.579504   
                                                   rel       6.476739   
rev_emo(best)                                      nominal   4.283888   
                                                   rel       5.792176   
clnv(ex)                                           nominal  12.857127   
                                                   rel      11.976237   
clnv(best)                                         nominal   5.702984   
                                                   rel       6.823883   
rev_clnv(ex)                                       nominal  12.690551   
                                                   rel      11.845428   
rev_clnv(best)                                     nominal   5.478272   
                                                   rel       6.658032   
trade_size(ex)->tick(all)                          nominal  -0.732363   
                                                   rel       1.106705   
trade_size(ex)->quote(best)                        nominal   1.223117   
                                                   rel       4.378642   
trade_size(ex)->quote(best)->quote(ex)             nominal   1.643041   
                                                   rel       4.716511   
quote(best)->quote(ex)                             nominal   7.179690   
                                                   rel       7.988814   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   1.643041   
                                                   rel       4.716511   
buy_sell                                           nominal   0.248330   
                                                   rel       2.509151   

TRADE_SIZE_binned                                               (1,3]  \
tick(all)                                          nominal   2.263843   
                                                   rel       2.695014   
tick(ex)                                           nominal   1.539866   
                                                   rel       1.404713   
quote(best)                                        nominal   5.872236   
                                                   rel       8.097964   
quote(ex)                                          nominal  16.596108   
                                                   rel      15.454604   
lr(ex)                                             nomina

In [95]:
eff_dfs[3]

year_binned                                                      2015  \
tick(all)                                          nominal   1.689467   
                                                   rel       2.364424   
tick(ex)                                           nominal   1.120487   
                                                   rel       1.282758   
quote(best)                                        nominal   6.167704   
                                                   rel       8.213946   
quote(ex)                                          nominal  14.743331   
                                                   rel      13.745120   
lr(ex)                                             nominal  14.743331   
                                                   rel      13.745120   
lr(best)                                           nominal   6.168639   
                                                   rel       8.306645   
rev_lr(ex)                                         nominal  14.743331   
                                                   rel      13.745120   
rev_lr(best)                                       nominal   6.175116   
                                                   rel       8.274235   
emo(ex)                                            nominal   3.797567   
                                                   rel       7.065330   
emo(best)                                          nominal   3.488564   
                                                   rel       6.344525   
rev_emo(ex)                                        nominal   3.748262   
                                                   rel       6.543983   
rev_emo(best)                                      nominal   3.532479   
                                                   rel       6.014626   
clnv(ex)                                           nominal   9.362185   
                                                   rel      10.891792   
clnv(best)                                         nominal   4.430882   
                                                   rel       7.062125   
rev_clnv(ex)                                       nominal   9.461577   
                                                   rel      10.764810   
rev_clnv(best)                                     nominal   4.766317   
                                                   rel       6.969890   
trade_size(ex)->tick(all)                          nominal  -0.651405   
                                                   rel      -0.126306   
trade_size(ex)->quote(best)                        nominal   1.984154   
                                                   rel       3.787129   
trade_size(ex)->quote(best)->quote(ex)             nominal   2.253384   
                                                   rel       4.036704   
quote(best)->quote(ex)                             nominal   6.479719   
                                                   rel       8.494478   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   2.253384   
                                                   rel       4.036704   
buy_sell                                           nominal   0.472800   
                                                   rel       2.566138   

year_binned                                                      2016  \
tick(all)                                          nominal   2.227762   
                                                   rel       2.297076   
tick(ex)                                           nominal   1.526681   
                                                   rel       0.890852   
quote(best)                                        nominal   5.713752   
                                                   rel       9.288466   
quote(ex)                                          nominal  15.738526   
                                                   rel      15.754051   
lr(ex)                                             nomina

In [96]:
eff_dfs[4]

ttm_binned                                                  ttm <= 1 month  \
tick(all)                                          nominal        1.967751   
                                                   rel            2.947536   
tick(ex)                                           nominal        1.707376   
                                                   rel            1.344665   
quote(best)                                        nominal        4.593420   
                                                   rel           11.231137   
quote(ex)                                          nominal       14.743711   
                                                   rel           19.914353   
lr(ex)                                             nominal       14.743711   
                                                   rel           19.914353   
lr(best)                                           nominal        4.647891   
                                                   rel           11.368485   
rev_lr(ex)                                         nominal       14.743711   
                                                   rel           19.914353   
rev_lr(best)                                       nominal        4.615776   
                                                   rel           11.306935   
emo(ex)                                            nominal        3.940607   
                                                   rel           10.669357   
emo(best)                                          nominal        3.540114   
                                                   rel            9.464564   
rev_emo(ex)                                        nominal        3.199804   
                                                   rel            9.763672   
rev_emo(best)                                      nominal        2.888794   
                                                   rel            8.849726   
clnv(ex)                                           nominal       10.570062   
                                                   rel           16.387132   
clnv(best)                                         nominal        4.134591   
                                                   rel           10.259368   
rev_clnv(ex)                                       nominal       10.428588   
                                                   rel           16.205727   
rev_clnv(best)                                     nominal        3.781262   
                                                   rel            9.968073   
trade_size(ex)->tick(all)                          nominal       -0.355484   
                                                   rel           -0.087986   
trade_size(ex)->quote(best)                        nominal        0.749074   
                                                   rel            5.417937   
trade_size(ex)->quote(best)->quote(ex)             nominal        1.121014   
                                                   rel            5.836459   
quote(best)->quote(ex)                             nominal        5.052892   
                                                   rel           11.720187   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal        1.121014   
                                                   rel            5.836459   
buy_sell                                           nominal        0.062872   
                                                   rel            4.957269   

ttm_binned                                                  ttm (1-2] month  \
tick(all)                                          nominal         1.744253   
                                                   rel             1.883314   
tick(ex)                                           nominal         0.977505   
                                                   rel             0.732443   
quote(best)                                        nominal         5.015198   
               

In [97]:
eff_dfs[5]

myn_binned                                                  myn <= 0.7  \
tick(all)                                          nominal    3.805546   
                                                   rel        6.745479   
tick(ex)                                           nominal    3.412220   
                                                   rel        3.001940   
quote(best)                                        nominal    6.490880   
                                                   rel       21.303594   
quote(ex)                                          nominal   20.132434   
                                                   rel       34.937756   
lr(ex)                                             nominal   20.132434   
                                                   rel       34.937756   
lr(best)                                           nominal    6.616625   
                                                   rel       21.575807   
rev_lr(ex)                                         nominal   20.132434   
                                                   rel       34.937756   
rev_lr(best)                                       nominal    6.533651   
                                                   rel       21.444588   
emo(ex)                                            nominal    7.559399   
                                                   rel       20.500381   
emo(best)                                          nominal    6.809843   
                                                   rel       18.747176   
rev_emo(ex)                                        nominal    6.479292   
                                                   rel       18.764575   
rev_emo(best)                                      nominal    5.888376   
                                                   rel       17.361260   
clnv(ex)                                           nominal   17.205601   
                                                   rel       29.815334   
clnv(best)                                         nominal    7.095162   
                                                   rel       20.098259   
rev_clnv(ex)                                       nominal   17.251897   
                                                   rel       29.546387   
rev_clnv(best)                                     nominal    6.516079   
                                                   rel       19.346554   
trade_size(ex)->tick(all)                          nominal   -0.311382   
                                                   rel       -1.063188   
trade_size(ex)->quote(best)                        nominal   -0.247823   
                                                   rel        6.910105   
trade_size(ex)->quote(best)->quote(ex)             nominal    0.181317   
                                                   rel        7.582852   
quote(best)->quote(ex)                             nominal    6.983733   
                                                   rel       22.143017   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal    0.181317   
                                                   rel        7.582852   
buy_sell                                           nominal   -0.660740   
                                                   rel        6.751363   

myn_binned                                                  myn (0.7-0.9]  \
tick(all)                                          nominal       2.478049   
                                                   rel           4.728458   
tick(ex)                                           nominal       2.256451   
                                                   rel           2.368627   
quote(best)                                        nominal       4.310322   
                                                   rel          17.706323   
quote(ex)                                          nominal      14.862295   
                                                   re

In [98]:
eff_dfs[6]

prox_q_binned                                               at mid   at quote  \
tick(all)                                          nominal     0.0   3.302376   
                                                   rel         0.0 -13.508220   
tick(ex)                                           nominal     0.0   0.969834   
                                                   rel         0.0 -28.836500   
quote(best)                                        nominal     0.0  15.003834   
                                                   rel         0.0  64.220468   
quote(ex)                                          nominal     0.0  15.404966   
                                                   rel         0.0  64.888421   
lr(ex)                                             nominal     0.0  15.404966   
                                                   rel         0.0  64.888421   
lr(best)                                           nominal     0.0  15.027650   
                                                   rel         0.0  64.096521   
rev_lr(ex)                                         nominal     0.0  15.404966   
                                                   rel         0.0  64.888421   
rev_lr(best)                                       nominal     0.0  14.982325   
                                                   rel         0.0  64.177458   
emo(ex)                                            nominal     0.0  15.404966   
                                                   rel         0.0  64.888421   
emo(best)                                          nominal     0.0  14.835896   
                                                   rel         0.0  63.169024   
rev_emo(ex)                                        nominal     0.0  15.404966   
                                                   rel         0.0  64.888421   
rev_emo(best)                                      nominal     0.0  14.783724   
                                                   rel         0.0  63.414274   
clnv(ex)                                           nominal     0.0  15.404926   
                                                   rel         0.0  64.888370   
clnv(best)                                         nominal     0.0  14.850701   
                                                   rel         0.0  63.181562   
rev_clnv(ex)                                       nominal     0.0  15.404845   
                                                   rel         0.0  64.888292   
rev_clnv(best)                                     nominal     0.0  14.812292   
                                                   rel         0.0  63.448535   
trade_size(ex)->tick(all)                          nominal     0.0  -7.261075   
                                                   rel         0.0 -28.020464   
trade_size(ex)->quote(best)                        nominal     0.0  -3.654344   
                                                   rel         0.0  23.404641   
trade_size(ex)->quote(best)->quote(ex)             nominal     0.0  -3.534408   
                                                   rel         0.0  23.627387   
quote(best)->quote(ex)                             nominal     0.0  15.312019   
                                                   rel         0.0  64.799849   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal     0.0  -3.534408   
                                                   rel         0.0  23.627387   
buy_sell                                           nominal     0.0  -2.970441   
                                                   rel         0.0  28.324667   

prox_q_binned                                                  inside  \
tick(all)                                          nominal   2.315420   
                                                   rel       3.854868   
tick(ex)                                           nominal   1.790926   
                                                   rel       3

In [99]:
master = pd.concat(eff_dfs, axis=1, keys=print_keys).T

In [100]:
master

tick(all)             tick(ex)             \
                                     nominal        rel   nominal        rel   
Option Type       C                 2.368956   2.787060  1.641275   1.216144   
                  P                 1.967647   2.166616  1.472416   0.923338   
Security Type     Index option      3.465315   0.707419  2.406263   0.604896   
                  Others            1.370533   1.004399  0.824128  -0.777969   
                  Stock options     2.488116   3.128149  1.846600   1.836602   
Trade Size        (0,1]             2.502912   2.898426  1.796003   1.660074   
                  (1,3]             2.263843   2.695014  1.539866   1.404713   
                  (3,5]             1.987521   2.220781  1.550745   0.956290   
                  (5,11]            1.946703   2.068593  1.467939   0.538272   
                  >11               1.828052   2.335737  1.223110   0.362687   
Year              2015              1.689467   2.364424  1.120487   1.282758   
                  2016              2.227762   2.297076  1.526681   0.890852   
                  2017              2.240243   3.010497  1.794266   1.452068   
Time to Maturity  ttm <= 1 month    1.967751   2.947536  1.707376   1.344665   
                  ttm (1-2] month   1.744253   1.883314  0.977505   0.732443   
                  ttm (2-3] month   2.107698   1.659584  1.168174   0.593640   
                  ttm (3-6] month   2.797238   1.647231  1.421605   0.609520   
                  ttm (6-12] month  2.843156   1.352037  1.293527   0.478670   
                  ttm > 12 month    4.784284   1.292006  1.764680   0.324753   
Moneyness         myn <= 0.7        3.805546   6.745479  3.412220   3.001940   
                  myn (0.7-0.9]     2.478049   4.728458  2.256451   2.368627   
                  myn (0.9-1.1]     1.860807   1.986301  1.216843   0.759298   
                  myn (1.1-1.3]     2.859158   0.470471  1.588582   0.234899   
                  myn > 1.3         3.812454   0.730733  2.704856   0.520800   
Location to Quote at mid            0.000000   0.000000  0.000000   0.000000   
                  at quote          3.302376 -13.508220  0.969834 -28.836500   
                  inside            2.315420   3.854868  1.790926   3.382516   
                  outside           4.051569   3.681179  1.694967  -3.325569   

                                   quote(best)             quote(ex)  \
                                       nominal        rel    nominal   
Option Type       C                   6.277463   8.687091  17.148075   
                  P                   5.130346   9.340715  15.446562   
Security Type     Index option       10.262529   2.358332  56.374215   
                  Others              4.259548  12.803767  10.495058   
                  Stock options       6.266108   7.563507  18.098609   
Trade Size        (0,1]               6.637340   7.585467  18.754145   
                  (1,3]               5.872236   8.097964  16.596108   
                  (3,5]               5.338059   8.503963  15.912070   
                  (5,11]              5.094794  11.515210  14.884577   
                  >11                 4.768014  10.966205  13.045129   
Year              2015                6.167704   8.213946  14.743331   
                  2016                5.713752   9.288466  15.738526   
                  2017                5.663834   8.570625  18.323413   
Time to Maturity  ttm <= 1 month      4.593420  11.231137  14.743711   
                  ttm (1-2] month     5.015198   5.289339  14.385087   
                  ttm (2-3] month     5.872983   4.423478  16.127663   
                  ttm (3-6] month     8.055202   4.521166  19.912735   
                  ttm (6-12] month    9.368054   3.710387  22.625065   
                  ttm > 12 month     16.879860   3.394719  32.557822   
Moneyness         myn <= 0.7          6.490880  21.303594  20.132434   
                  myn (0.7-0.9]       4.310322  17

In [117]:
master.iloc[:,0:20]

tick(all)             tick(ex)             \
                                     nominal        rel   nominal        rel   
Option Type       C                 2.368956   2.787060  1.641275   1.216144   
                  P                 1.967647   2.166616  1.472416   0.923338   
Security Type     Index option      3.465315   0.707419  2.406263   0.604896   
                  Others            1.370533   1.004399  0.824128  -0.777969   
                  Stock options     2.488116   3.128149  1.846600   1.836602   
Trade Size        (0,1]             2.502912   2.898426  1.796003   1.660074   
                  (1,3]             2.263843   2.695014  1.539866   1.404713   
                  (3,5]             1.987521   2.220781  1.550745   0.956290   
                  (5,11]            1.946703   2.068593  1.467939   0.538272   
                  >11               1.828052   2.335737  1.223110   0.362687   
Year              2015              1.689467   2.364424  1.120487   1.282758   
                  2016              2.227762   2.297076  1.526681   0.890852   
                  2017              2.240243   3.010497  1.794266   1.452068   
Time to Maturity  ttm <= 1 month    1.967751   2.947536  1.707376   1.344665   
                  ttm (1-2] month   1.744253   1.883314  0.977505   0.732443   
                  ttm (2-3] month   2.107698   1.659584  1.168174   0.593640   
                  ttm (3-6] month   2.797238   1.647231  1.421605   0.609520   
                  ttm (6-12] month  2.843156   1.352037  1.293527   0.478670   
                  ttm > 12 month    4.784284   1.292006  1.764680   0.324753   
Moneyness         myn <= 0.7        3.805546   6.745479  3.412220   3.001940   
                  myn (0.7-0.9]     2.478049   4.728458  2.256451   2.368627   
                  myn (0.9-1.1]     1.860807   1.986301  1.216843   0.759298   
                  myn (1.1-1.3]     2.859158   0.470471  1.588582   0.234899   
                  myn > 1.3         3.812454   0.730733  2.704856   0.520800   
Location to Quote at mid            0.000000   0.000000  0.000000   0.000000   
                  at quote          3.302376 -13.508220  0.969834 -28.836500   
                  inside            2.315420   3.854868  1.790926   3.382516   
                  outside           4.051569   3.681179  1.694967  -3.325569   

                                   quote(best)             quote(ex)  \
                                       nominal        rel    nominal   
Option Type       C                   6.277463   8.687091  17.148075   
                  P                   5.130346   9.340715  15.446562   
Security Type     Index option       10.262529   2.358332  56.374215   
                  Others              4.259548  12.803767  10.495058   
                  Stock options       6.266108   7.563507  18.098609   
Trade Size        (0,1]               6.637340   7.585467  18.754145   
                  (1,3]               5.872236   8.097964  16.596108   
                  (3,5]               5.338059   8.503963  15.912070   
                  (5,11]              5.094794  11.515210  14.884577   
                  >11                 4.768014  10.966205  13.045129   
Year              2015                6.167704   8.213946  14.743331   
                  2016                5.713752   9.288466  15.738526   
                  2017                5.663834   8.570625  18.323413   
Time to Maturity  ttm <= 1 month      4.593420  11.231137  14.743711   
                  ttm (1-2] month     5.015198   5.289339  14.385087   
                  ttm (2-3] month     5.872983   4.423478  16.127663   
                  ttm (3-6] month     8.055202   4.521166  19.912735   
                  ttm (6-12] month    9.368054   3.710387  22.625065   
                  ttm > 12 month     16.879860   3.394719  32.557822   
Moneyness         myn <= 0.7          6.490880  21.303594  20.132434   
                  myn (0.7-0.9]       4.310322  17

In [118]:
master.iloc[:,21:40]

rev_emo(ex) rev_emo(best)             \
                                           rel       nominal        rel   
Option Type       C                   7.694203      4.300934   7.000756   
                  P                   8.105171      3.422486   7.451281   
Security Type     Index option        1.214250      5.524196   1.221930   
                  Others             11.514777      2.751397  11.089187   
                  Stock options       6.531479      4.321332   5.745770   
Trade Size        (0,1]               6.476739      4.283888   5.792176   
                  (1,3]               6.960860      3.939399   6.265472   
                  (3,5]               7.333065      3.707130   6.668206   
                  (5,11]             10.392813      3.680045   9.713673   
                  >11                 9.981505      3.398195   9.350753   
Year              2015                6.543983      3.532479   6.014626   
                  2016                8.088080      3.693256   7.485005   
                  2017                7.872927      4.469888   6.982067   
Time to Maturity  ttm <= 1 month      9.763672      2.888794   8.849726   
                  ttm (1-2] month     4.648332      3.748884   4.430100   
                  ttm (2-3] month     4.069327      4.775144   3.856754   
                  ttm (3-6] month     4.240257      6.142963   4.063048   
                  ttm (6-12] month    3.559283      7.049135   3.384851   
                  ttm > 12 month      3.220166     10.943618   3.107813   
Moneyness         myn <= 0.7         18.764575      5.888376  17.361260   
                  myn (0.7-0.9]      15.411242      3.711672  14.111065   
                  myn (0.9-1.1]       6.334405      3.495061   5.776213   
                  myn (1.1-1.3]       1.123007      5.447760   1.076903   
                  myn > 1.3           1.387311      7.281351   1.169914   
Location to Quote at mid              0.000000      0.000000   0.000000   
                  at quote           64.888421     14.783724  63.414274   
                  inside              4.603336      3.356854   3.896249   
                  outside            20.063230     12.595844  19.492503   

                                     clnv(ex)            clnv(best)  \
                                      nominal        rel    nominal   
Option Type       C                 12.215973  13.033426   5.545472   
                  P                 10.947257  13.555627   4.482408   
Security Type     Index option      18.198776   4.135419   6.693394   
                  Others             7.754844  15.131944   3.496897   
                  Stock options     13.077245  12.674333   5.645131   
Trade Size        (0,1]             12.857127  11.976237   5.702984   
                  (1,3]             11.684667  12.441712   5.180638   
                  (3,5]             11.315871  12.778236   4.729606   
                  (5,11]            10.991318  15.802087   4.651369   
                  >11                9.942347  14.956485   4.224841   
Year              2015               9.362185  10.891792   4.430882   
                  2016              11.036555  12.950356   4.977291   
                  2017              13.747536  14.842612   5.424162   
Time to Maturity  ttm <= 1 month    10.570062  16.387132   4.134591   
                  ttm (1-2] month    9.983341   7.739411   4.392357   
                  ttm (2-3] month   11.308705   6.725458   5.281856   
                  ttm (3-6] month   14.243241   7.338446   6.970565   
                  ttm (6-12] month  16.195550   6.338262   8.085447   
                  ttm > 12 month    22.069182   5.473627  13.588732   
Moneyness         myn <= 0.7        17.205601  29.815334   7.095162   
                  myn (0.7-0.9]     12.018693  24.265514   4.569527   
                  myn (0.9-1.1]     10.313319  11.072547   4.256872   
                  myn (1.1-1.3]     15.329095   2.706771   9.192966   
         

In [119]:
master.iloc[:,41:-1]

trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex)  \
                                                                                  rel   
Option Type       C                                                          4.309526   
                  P                                                          4.247147   
Security Type     Index option                                               1.749184   
                  Others                                                     6.640845   
                  Stock options                                              3.372486   
Trade Size        (0,1]                                                      4.716511   
                  (1,3]                                                      5.001148   
                  (3,5]                                                      4.964571   
                  (5,11]                                                     3.407876   
                  >11                                                        3.003016   
Year              2015                                                       4.036704   
                  2016                                                       4.267766   
                  2017                                                       4.395338   
Time to Maturity  ttm <= 1 month                                             5.836459   
                  ttm (1-2] month                                            2.088945   
                  ttm (2-3] month                                            1.218822   
                  ttm (3-6] month                                            0.867671   
                  ttm (6-12] month                                           0.451950   
                  ttm > 12 month                                             0.255260   
Moneyness         myn <= 0.7                                                 7.582852   
                  myn (0.7-0.9]                                              8.900576   
                  myn (0.9-1.1]                                              3.490561   
                  myn (1.1-1.3]                                              0.317534   
                  myn > 1.3                                                  0.236676   
Location to Quote at mid                                                     0.000000   
                  at quote                                                  23.627387   
                  inside                                                     3.475950   
                  outside                                                    4.837371   

                                    buy_sell  
                                     nominal  
Option Type       C                 0.468056  
                  P                 0.529072  
Security Type     Index option      3.341933  
                  Others            0.216929  
                  Stock options     0.563277  
Trade Size        (0,1]             0.248330  
                  (1,3]             0.445001  
                  (3,5]             0.254992  
                  (5,11]            0.839305  
                  >11               0.905394  
Year              2015              0.472800  
                  2016              0.766071  
                  2017             -0.120827  
Time to Maturity  ttm <= 1 month    0.062872  
                  ttm (1-2] month   0.857769  
                  ttm (2-3] month   0.709331  
                  ttm (3-6] month   1.582060  
                  ttm (6-12] month  1.520536  
                  ttm > 12 month    2.932733  
Moneyness         myn <= 0.7       -0.660740  
                  myn (0.7-0.9]     0.415172  
                  myn (0.9-1.1]     0.950240  
                  myn (1.1-1.3]    -1.409448  
                  myn > 1.3        -2.793827  
Location to Quote at mid            0.000000  
                  at quote         -2.970441  
                  inside            0.880201  
 

In [120]:
master.style.pipe(set_tex_style, caption=("master-short","master-long"), label=f"{key}-master-eff-spread")